In [2]:
%pylab
import math
from scipy import ndimage
import numpy as np

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [22]:
image = ndimage.imread('./lighthouse_RAW_noisy_sigma0.01.png.edgedirected.mine.png')

# source: http://www.fourcc.org/fccyvrgb.php
def toYCrCb(image):
    YRCoeff = 0.257
    YGCoeff = 0.504
    YBCoeff = 0.098
    YConstant = 16
    
    CrRCoeff = 0.439
    CrGCoeff = 0.368
    CrBCoeff = 0.071
    CrConstant = 128
    
    CbRCoeff = 0.148
    CbGCoeff = 0.291
    CbBCoeff = 0.439
    CbConstant = 128
    
    reds = image[:, :, 0]
    greens = image[:, :, 1]
    blues = image[:, :, 2]
    
    Y  = YRCoeff * reds + YGCoeff * greens + YBCoeff * blues + YConstant
    Cr = CrRCoeff * reds - CrGCoeff * greens - CrBCoeff * blues + CrConstant
    Cb = -(CbRCoeff * reds) - CbGCoeff * greens + CbBCoeff * blues + CbConstant
    
    result = np.empty(shape=(image.shape[0], image.shape[1], 3))
    result[:, :, 0] = Y
    result[:, :, 1] = Cr
    result[:, :, 2] = Cb
    
    return result

def toRGB(image):
    YCoeff = 1.164
    
    BCbCoeff = 2.018
    
    GCrCoeff = -0.813
    GCbCoeff = -0.391
    
    RCrCoeff = 1.596
    
    Y = image[:, :, 0]
    Cr = image[:, :, 1]
    Cb = image[:, :, 2]
    
    Y -= 16
    Cr -= 128
    Cb -= 128
    
    YPiece = Y * YCoeff
    blues = YPiece + BCbCoeff * Cb
    greens = YPiece + (GCrCoeff * Cr) + (GCbCoeff * Cb)
    reds = YPiece + (RCrCoeff * Cr)
    
    result = np.empty(shape=(image.shape[0], image.shape[1], 3))
    
    overflowedReds = reds > 255
    overflowedGreens = greens > 255
    overflowedBlues = blues > 255
    
    underflowedReds = reds < 0
    underflowedGreens = greens < 0
    underflowedBlues = blues < 0
    
    reds[overflowedReds] = 255.
    greens[overflowedGreens] = 255.
    blues[overflowedBlues] = 255.
    
    reds[underflowedReds] = 0.
    greens[underflowedGreens] = 0.
    blues[underflowedBlues] = 0.
    
    result[:, :, 0] = reds
    result[:, :, 1] = greens
    result[:, :, 2] = blues
    
    return result

def naiveMedianFilter(window):
    sortedWindow = sorted(window)
    return sortedWindow[len(sortedWindow) / 2]

def applyGamma(image, gamma = 1/2.2):
    red = image[:, :, 0]
    green = image[:, :, 1]
    blue = image[:, :, 2]
    gamma = 1 / 2.2

    red = np.power(red, gamma)
    green = np.power(green, gamma)
    blue = np.power(blue, gamma)

    result = np.empty(shape=(image.shape[0], image.shape[1], 3))
    result[:, :, 0] = red
    result[:, :, 1] = green
    result[:, :, 2] = blue
    
    return result

yuvImage = toYCrCb(image)
# naive usage:
# yuvImage[:, :, 1] = ndimage.filters.generic_filter(yuvImage[:, :, 1], size=10, function=naiveMedianFilter)
# yuvImage[:, :, 2] = ndimage.filters.generic_filter(yuvImage[:, :, 2], size=10, function=naiveMedianFilter)

yuvImage[:, :, 1] = ndimage.filters.median_filter(yuvImage[:, :, 1], size=10)
yuvImage[:, :, 2] = ndimage.filters.median_filter(yuvImage[:, :, 2], size=10)
rgbImage = toRGB(yuvImage)

imshow(applyGamma(rgbImage / 255.0))